In [ ]:
!pip install import-ipynb
import import_ipynb
import util as u
import pandas as pd
import numpy as np
import requests as rq
import xml.etree.ElementTree as ET
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
def get_materias_legislatura_atual():
    url = "https://legis.senado.leg.br/dadosabertos/materia/legislaturaatual"
    df = u.parse_XML(url,["DescricaoIdentificacaoMateria","CodigoMateria","IndicadorTramitando"],"Materias/Materia/IdentificacaoMateria")
    df.rename(columns = {'DescricaoIdentificacaoMateria' : 'Identificacao'}, inplace = True)
    df.rename(columns = {'IndicadorTramitando' : 'Tramitando'}, inplace = True)
    df.set_index('CodigoMateria')
    return df

In [ ]:
def get_materias_tramitando():
    url = "https://legis.senado.leg.br/dadosabertos/materia/tramitando"
    df = u.parse_XML(url,["DescricaoIdentificacaoMateria","CodigoMateria","IndicadorTramitando"],"Materias/Materia/IdentificacaoMateria")
    df.rename(columns = {'DescricaoIdentificacaoMateria' : 'Identificacao'}, inplace = True)
    df.rename(columns = {'IndicadorTramitando' : 'Tramitando'}, inplace = True)
    df.set_index('CodigoMateria')
    return df

In [ ]:
def get_detalhes_materia(codigo_materia):
    url = "https://legis.senado.leg.br/dadosabertos/materia/%d" % codigo_materia
    df = u.parse_XML(url,["IdentificacaoMateria/DescricaoIdentificacaoMateria","DadosBasicosMateria/EmentaMateria","DadosBasicosMateria/IndexacaoMateria","DadosBasicosMateria/NaturezaMateria/DescricaoNatureza","Classificacoes/Classificacao/DescricaoClasse","Classificacoes/Classificacao/DescricaoClasseHierarquica"],"Materia")
    df.rename(columns = {'IdentificacaoMateria/DescricaoIdentificacaoMateria' : 'Identificacao'}, inplace = True)
    df.rename(columns = {'DadosBasicosMateria/EmentaMateria' : 'Ementa'}, inplace = True)
    df.rename(columns = {'DadosBasicosMateria/IndexacaoMateria' : 'Indexacao'}, inplace = True)
    df.rename(columns = {'DadosBasicosMateria/NaturezaMateria/DescricaoNatureza' : 'Natureza'}, inplace = True)
    df.rename(columns = {'Classificacoes/Classificacao/DescricaoClasse' : 'Classe'}, inplace = True)
    df.rename(columns = {'Classificacoes/Classificacao/DescricaoClasseHierarquica' : 'ClasseHierarquica'}, inplace = True)
    df['CodigoMateria'] = codigo_materia
    df.set_index('CodigoMateria')
    return df

In [ ]:
def get_emendas_materia(codigo_materia):
    url = "https://legis.senado.leg.br/dadosabertos/materia/emendas/%d" % codigo_materia    
    df = u.parse_XML(url,["AutoriaEmenda/Autor/IdentificacaoParlamentar/CodigoParlamentar", 'DataApresentacao'],"Materia/Emendas/Emenda")
    df.rename(columns = {'AutoriaEmenda/Autor/IdentificacaoParlamentar/CodigoParlamentar' : 'CodigoParlamentar'}, inplace = True)        
    df['DataApresentacao'] = pd.to_datetime(df['DataApresentacao'])
    df['CodigoMateria'] = codigo_materia
    df = df.set_index('CodigoMateria')
    return df

In [ ]:
def isMateriaTramitando(CodigoMateria,DataFrameMateriasTramitando):
    if (CodigoMateria in DataFrameMateriasTramitando.CodigoMateria.values):
        return "Sim"
    else:
        return "Não"

### Recupera as matérias da legislatura atual

In [ ]:
materias_legislatura_atual = get_materias_legislatura_atual()
materias_legislatura_atual = materias_legislatura_atual.set_index('CodigoMateria')

In [ ]:
#A coluna "PegouDetalhe" servirá para controlar o preenchimento dos detalhes
materias_legislatura_atual.insert(1,"PegouDetalhe",False)

In [ ]:
# Considera apenas as matérias que estejam tramitando no momneto
materias_legislatura_atual = materias_legislatura_atual.query('Tramitando == "Sim"')

In [ ]:
len(materias_legislatura_atual)

### Busca os detalhes de cada matéria

In [ ]:
df_detalhes_materias = pd.DataFrame()

In [ ]:
materias_aux = df_materias_pesquisar.loc[df_materias_pesquisar['PegouDetalhe'] == False]
for codigo, row in materias_aux[0:2000].iterrows():    
    try:
        df_detalhe = get_detalhes_materia(int(codigo))
        df_detalhes_materias = pd.concat([df_detalhes_materias,df_detalhe])
        df_materias_pesquisar.at[codigo, 'PegouDetalhe'] = True    
    except TimeoutError:
        print ("não achei %s" % codigo)

In [ ]:
df_detalhes_materias

In [ ]:
df_materias_pesquisar.loc[df_materias_pesquisar['PegouDetalhe'] == True]

In [ ]:
df_materias_pesquisar.loc[df_materias_pesquisar['PegouDetalhe'] == False]

In [ ]:
materias_legislatura_atual.set_index('CodigoMateria')

In [ ]:
materias_legislatura_atual['PegouDetalhe'] = False

# Atualização das matérias

In [ ]:
df_detalhes_materias = pd.read_csv("df_detalhes_materias.csv",sep=",")
df_materias_legislatura_atual = get_materias_legislatura_atual()
df_materias_legislatura_atual = df_materias_legislatura_atual.set_index('CodigoMateria')
df_materias_legislatura_atual = df_materias_legislatura_atual.query('Tramitando == "Sim"')

In [ ]:
len(df_detalhes_materias)

In [ ]:
df_detalhes_materias.head(1)

In [ ]:
df_materias_legislatura_atual.head(1)

In [ ]:
len(df_materias_legislatura_atual)

In [ ]:
#Filtrando os que já existem na lista    
codigo_materia_lista = df_detalhes_materias['CodigoMateria'].tolist()    
lista_strings = [str(valor) for valor in codigo_materia_lista]
mascara = ~df_materias_legislatura_atual.index.isin(lista_strings)
df_filtrado = df_materias_legislatura_atual[mascara]
df_materias_pesquisar = pd.DataFrame(df_filtrado)
df_materias_pesquisar.insert(1,"PegouDetalhe",False)

In [ ]:
#Quantidade de novas matérias para adicionar
len(df_materias_pesquisar)

In [ ]:
df_materias_pesquisar.head(5)

In [ ]:
materias_aux = df_materias_pesquisar.loc[df_materias_pesquisar['PegouDetalhe'] == False]
for codigo, row in materias_aux.iterrows():    
    try:
        df_detalhe = get_detalhes_materia(int(codigo))
        df_detalhes_materias = pd.concat([df_detalhes_materias,df_detalhe])
        df_materias_pesquisar.at[codigo, 'PegouDetalhe'] = True    
    except TimeoutError:
        print ("não achei %s" % codigo)

In [ ]:
len(df_materias_pesquisar.loc[df_materias_pesquisar['PegouDetalhe'] == False])

In [ ]:
len(df_materias_pesquisar.loc[df_materias_pesquisar['PegouDetalhe'] == True])

In [ ]:
df_detalhes_materias.to_csv('df_detalhes_materias.csv',index = False)

# Atualização das matérias de interesse

In [ ]:
df_detalhes_materias = pd.read_csv("df_detalhes_materias.csv",sep=",")
df_combinado = pd.read_csv('interesse.csv')
mascara = ~df_combinado['CodigoMateria'].isin(df_detalhes_materias['CodigoMateria'])
df_filtrado = df_combinado[mascara]
df_materias_pesquisar = pd.DataFrame(df_filtrado)
df_materias_pesquisar.insert(1,"PegouDetalhe",False)
df_materias_pesquisar = df_materias_pesquisar.set_index('CodigoMateria')
#Remeve as duplicatas
df_materias_pesquisar = df_materias_pesquisar[~df_materias_pesquisar.index.duplicated(keep='first')]

In [ ]:
#Quantidade de novas matérias para adicionar
len(df_materias_pesquisar)

In [ ]:
materias_aux = df_materias_pesquisar.loc[df_materias_pesquisar['PegouDetalhe'] == False]
for codigo, row in materias_aux.iterrows():    
    try:        
        df_detalhe = get_detalhes_materia(int(codigo))
        df_detalhes_materias = pd.concat([df_detalhes_materias,df_detalhe])
        df_materias_pesquisar.at[codigo, 'PegouDetalhe'] = True    
    except TimeoutError:
        print ("não achei %s" % codigo)

In [ ]:
len(df_materias_pesquisar.loc[df_materias_pesquisar['PegouDetalhe'] == False])

In [ ]:
len(df_materias_pesquisar.loc[df_materias_pesquisar['PegouDetalhe'] == True])

In [ ]:
df_detalhes_materias.to_csv('df_detalhes_materias.csv',index = False)


In [ ]:
len(df_detalhes_materias)

# Limpando a df_detalhes_materias

In [ ]:
df_detalhes_materias = pd.read_csv("df_detalhes_materias.csv",sep=",")
# Juntando os CodigoMateria das matérias da legislatura atual e das matérias de interesse
df_materias_legislatura_atual = get_materias_legislatura_atual()
df_materias_legislatura_atual = df_materias_legislatura_atual.query('Tramitando == "Sim"')
df_materias_legislatura_atual['CodigoMateria'] = df_materias_legislatura_atual['CodigoMateria'].astype(int)
df_interesse_combinado = pd.read_csv('interesse.csv')


In [ ]:
df_materias_legislatura_atual

In [ ]:
df_interesse_combinado

In [ ]:
mask1 = df_detalhes_materias['CodigoMateria'].isin(df_materias_legislatura_atual['CodigoMateria'])
mask2 = df_detalhes_materias['CodigoMateria'].isin(df_interesse_combinado['CodigoMateria'])
mascara_final =  (mask1 | mask2)
#df_detalhes_materias_new = df_detalhes_materias[mascara_final]
df_detalhes_materias = df_detalhes_materias[mascara_final]

In [ ]:
len(df_detalhes_materias)

In [ ]:
len(df_detalhes_materias_new)

In [ ]:
df_detalhes_materias[~df_detalhes_materias['CodigoMateria'].isin(df_detalhes_materias_new['CodigoMateria'])].to_csv('materias_excluir.csv')

In [ ]:
df_detalhes_materias.head(5)

In [ ]:
df_detalhes_materias = df_detalhes_materias.drop_duplicates(subset='CodigoMateria', keep='first')
#df_detalhes_materias = df_detalhes_materias.drop(columns=['Unnamed: 0.5','Unnamed: 0.4','Unnamed: 0.3','Unnamed: 0.2','Unnamed: 0.1','Unnamed: 0' ])

In [ ]:
df_detalhes_materias.to_csv('df_detalhes_materias.csv',index = False)

In [ ]:
df_detalhes_materias

# Alimentando a coluna "tramitando"

In [ ]:
df_detalhes_materias = pd.read_csv("df_detalhes_materias.csv")

In [ ]:
df_detalhes_materias.head(2)

In [ ]:
len(df_detalhes_materias)

In [ ]:
df_materias_tramitando = get_materias_tramitando()
df_materias_tramitando['CodigoMateria'] = df_materias_tramitando['CodigoMateria'].astype(int)

In [ ]:
len(df_materias_tramitando)

In [ ]:
df_detalhes_materias['Tramitando'] = df_detalhes_materias.apply(
    lambda row: isMateriaTramitando(row['CodigoMateria'],df_materias_tramitando),axis = 1)

In [ ]:
df_detalhes_materias

In [ ]:
df_detalhes_materias.to_csv('df_detalhes_materias.csv',index = False)

## Pegando as emendas das matérias

In [ ]:
df_materias_pesquisar = pd.DataFrame(df_detalhes_materias)
df_materias_pesquisar['PegouDetalhe'] = False
df_emendas = pd.DataFrame()    

In [ ]:
materias_aux = df_materias_pesquisar.loc[df_materias_pesquisar['PegouDetalhe'] == False]
for codigo, row in materias_aux.iterrows():    
    codigo_materia = row['CodigoMateria']    
    df_emendas_da_materia = get_emendas_materia(int(codigo_materia))    
    df_emendas = pd.concat([df_emendas,df_emendas_da_materia])    
    df_materias_pesquisar.at[codigo, 'PegouDetalhe'] = True    

In [ ]:
len(df_materias_pesquisar.loc[df_materias_pesquisar['PegouDetalhe'] == False])

In [ ]:
len(df_materias_pesquisar.loc[df_materias_pesquisar['PegouDetalhe'] == True])

In [ ]:
len(df_emendas)

In [ ]:
df_emendas = df_emendas.reset_index()

In [ ]:
df_emendas.to_csv('df_emendas.csv',index = False)

In [ ]:
df_emendas